# 02_preprocessing_variants_complete

Vollständige konsolidierte Erstellung zweier Preprocessing-Varianten:

1. **raw_minimal** – minimale Normalisierung (nur lowercase + Whitespace).
2. **clean_lemma_stop** – vollständige Pipeline (Normalisierung, Acronym-Expansion, Stopwords, optional Lemmatization).

Ausgabe CSV-Dateien (Ordner `data/processed/`):
- `cves_processed_text_raw.csv`
- `cves_processed_text_clean.csv`

Inhalte direkt aus legacy Notebook übernommen – keine neuen Features hinzugefügt.

## 1. Imports & Einstellungen

In [4]:
import re, html
from pathlib import Path
import pandas as pd
import numpy as np
import nltk
from collections import Counter

USE_LEMMATIZATION = True  # optional (clean Variante)

# Stopwords sicherstellen
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# spaCy optional
try:
    import spacy
    try:
        nlp = spacy.load('en_core_web_sm', disable=['parser','ner']) if USE_LEMMATIZATION else None
    except OSError:
        print('Hinweis: en_core_web_sm nicht installiert')
        nlp = None
except ImportError:
    if USE_LEMMATIZATION:
        print('spaCy nicht installiert – Lemmatisierung deaktiviert')
    nlp = None

DATA_DIR = Path('..') / 'data' / 'raw'
DEFAULT_CSV_LIST = ['cves_v40.csv','cves_v31.csv','cves_v30.csv','cves_v2.csv']
CSV_SOURCE = 'cves_v30.csv'

if CSV_SOURCE is None:
    candidates = DEFAULT_CSV_LIST
elif isinstance(CSV_SOURCE, str):
    candidates = [CSV_SOURCE]
elif isinstance(CSV_SOURCE, (list, tuple)):
    candidates = list(CSV_SOURCE)
else:
    raise ValueError('CSV_SOURCE muss None, str oder Liste sein')
CSV_PATH = next((DATA_DIR / c for c in candidates if (DATA_DIR / c).exists()), None)
print({'candidates': candidates, 'CSV_PATH': str(CSV_PATH)})
assert CSV_PATH is not None, 'Keine CVE CSV gefunden.'

{'candidates': ['cves_v30.csv'], 'CSV_PATH': '../data/raw/cves_v30.csv'}


## 2. Laden der Daten

In [5]:
USE_COLS = ['cve_id','severity','description']
df = pd.read_csv(CSV_PATH, usecols=USE_COLS)
print('Zeilen eingelesen:', len(df))
print(df.head(3)[USE_COLS])
df = df.dropna(subset=['description'])
print('Nach Drop NA:', len(df))

Zeilen eingelesen: 53856
          cve_id  severity                                        description
0  CVE-2000-0258      HIGH  IIS 4.0 and 5.0 allows remote attackers to cau...
1  CVE-2004-0847  CRITICAL  The Microsoft .NET forms authentication capabi...
2  CVE-2005-0109    MEDIUM  Hyper-Threading technology, as used in FreeBSD...
Nach Drop NA: 53856


## 3. Acronym Expansion Mapping

In [6]:
ACRONYM_MAP = {
    # Impact-/Exploit-Typen
    'rce': 'remote code execution',
    'codeexec': 'remote code execution',
    'eop': 'elevation of privilege',
    'privesc': 'privilege escalation',
    'pe': 'privilege escalation',
    'lpe': 'local privilege escalation',
    'uep': 'unauthorized privilege escalation',
    'dos': 'denial of service',
    'crash': 'denial of service',
    'ddos': 'distributed denial of service',
    'infoleak': 'information disclosure',
    'id': 'information disclosure',
    'leak': 'information disclosure',

    # Speicher-/Memory-Corruption
    'bof': 'buffer overflow',
    'bo': 'buffer overflow',
    'heapof': 'heap overflow',
    'heapovf': 'heap overflow',
    'heapoverflow': 'heap overflow',
    'stackof': 'stack overflow',
    'stackoverflow': 'stack overflow',
    'oob': 'out of bounds access',
    'oobr': 'out of bounds read',
    'oobw': 'out of bounds write',
    'iob': 'index out of bounds',
    'uaf': 'use after free',
    'df': 'double free',
    'npd': 'null pointer dereference',
    'tc': 'type confusion',
    'typeconf': 'type confusion',
    'fmtstr': 'format string vulnerability',
    'race': 'race condition',
    'toctou': 'time of check to time of use',
    'intof': 'integer overflow',
    'intuf': 'integer underflow',
    'sigrop': 'signal oriented programming',

    # Web-/AppSec-Klassen
    'xss': 'cross site scripting',
    'xxs': 'cross site scripting',
    'uxss': 'universal cross site scripting',
    'csrf': 'cross site request forgery',
    'xsrf': 'cross site request forgery',
    'ssrf': 'server side request forgery',
    'sqli': 'sql injection',
    'nosqli': 'nosql injection',
    'ldapi': 'ldap injection',
    'ognl': 'ognl expression injection',
    'cmdi': 'command injection',
    'rce-inj': 'command injection',
    'ssti': 'server side template injection',
    'xssi': 'cross site script inclusion',
    'xxe': 'xml external entity',
    'xee': 'xml external entity',
    'lfi': 'local file inclusion',
    'rfi': 'remote file inclusion',
    'idor': 'insecure direct object reference',
    'bidor': 'blind insecure direct object reference',
    'dirtrav': 'directory traversal',
    'pathtrav': 'directory traversal',
    'openredirect': 'open redirect',
    'or': 'open redirect',
    'crlfi': 'crlf injection',
    'hpp': 'http parameter pollution',
    'hhi': 'host header injection',
    'reqsmuggle': 'http request smuggling',
    'hrs': 'http request smuggling',
    'reqsplit': 'http response splitting',
    'cachepoison': 'web cache poisoning',
    'deser': 'insecure deserialization',
    'insecdeser': 'insecure deserialization',
    'proto_pollution': 'prototype pollution',
    'fileupload': 'insecure file upload',
    'zipSlip': 'zip slip path traversal',

    # AuthN/AuthZ/Session
    'ato': 'account takeover',
    '2fa': 'two factor authentication',
    'mfa': 'multi factor authentication',
    'sso': 'single sign on',
    'saml': 'security assertion markup language',
    'oauth': 'oauth',
    'oidc': 'openid connect',
    'jwt': 'json web token',
    'jwts': 'json web token',
    'jwk': 'json web key',
    'jwks': 'json web key set',
    'pkce': 'proof key for code exchange',
    'bruteforce': 'credential brute force',
    'credstuff': 'credential stuffing',

    # Protokolle/HTTP-Sonderfälle
    'wsx': 'websocket hijacking',
    'dnsrb': 'dns rebinding',
    'ssrf-blind': 'blind server side request forgery',

    # Plattform-/Mitigations-/Exploitation-Techniques
    'aslr': 'address space layout randomization',
    'kaslr': 'kernel address space layout randomization',
    'dep': 'data execution prevention',
    'nx': 'no execute',
    'rop': 'return oriented programming',
    'jop': 'jump oriented programming',
    'cfi': 'control flow integrity',
    'cet': 'control-flow enforcement technology',
    'pie': 'position independent executable',
    'relro': 'relocation read-only',
    'pac': 'pointer authentication',
    'sandbox': 'process sandboxing',
    'seccomp': 'secure computing mode',
    'fortify': 'fortify source hardening',

    # Cloud/Config/Secrets
    'misconfig': 'security misconfiguration',
    'openbucket': 'public cloud storage misconfiguration',
    's3public': 'public cloud storage misconfiguration',
    'imds': 'cloud instance metadata service exposure',
    'k8srbac': 'kubernetes rbac misconfiguration',
    'secretleak': 'secrets exposure',
    'hardcodedcred': 'hardcoded credentials',

    # Kataloge/Scoring/Taxonomien
    'cve': 'common vulnerabilities and exposures',
    'cwe': 'common weakness enumeration',
    'cvss': 'common vulnerability scoring system',
    'epss': 'exploit prediction scoring system',
    'cpe': 'common platform enumeration',

    # Sonstiges/Meta
    'poc': 'proof of concept',
    'exploit': 'exploit',
    '0day': 'zero day',
    'zeroday': 'zero day',
    'nday': 'n day',
}
ACRONYM_REGEX = re.compile(r'\b(' + '|'.join(map(re.escape, ACRONYM_MAP.keys())) + r')\b', re.IGNORECASE)
def expand_acronyms(text: str) -> str:
    def repl(m):
        return ACRONYM_MAP.get(m.group(1).lower(), m.group(1))
    return ACRONYM_REGEX.sub(repl, text)
print(expand_acronyms('RCE via SSRF causes LPE + XSS & DoS'))

remote code execution via server side request forgery causes local privilege escalation + cross site scripting & denial of service


## 4. Preprocessing-Funktionen (clean Variante)

In [7]:
PUNCT_REGEX = re.compile(r"[\"'`´’“”‘()\[\]{}<>=:;,+*/\\|~^]")
MULTI_WS = re.compile(r'\s{2,}')
HTML_TAG = re.compile(r'<[^>]+>')
DIGITS = re.compile(r'\b\d+\b')
NON_ALPHANUM = re.compile(r'[^a-z0-9 ]')

def normalize_basic(text: str) -> str:
    text = text.lower()
    text = html.unescape(text)
    text = HTML_TAG.sub(' ', text)
    text = text.replace('\t',' ').replace('\n',' ')
    text = PUNCT_REGEX.sub(' ', text)
    text = DIGITS.sub(' ', text)
    text = NON_ALPHANUM.sub(' ', text)
    text = MULTI_WS.sub(' ', text).strip()
    return text

def remove_stopwords(tokens):
    return [t for t in tokens if t not in STOPWORDS and len(t) > 1]

def lemmatize_tokens(tokens):
    if not USE_LEMMATIZATION or nlp is None:
        return tokens
    doc = nlp(' '.join(tokens))
    return [tok.lemma_ for tok in doc]

def preprocess_text(text: str) -> str:
    text = normalize_basic(text)
    text = expand_acronyms(text)
    tokens = text.split(' ')
    tokens = remove_stopwords(tokens)
    tokens = lemmatize_tokens(tokens)
    return ' '.join(tokens)

print(preprocess_text('This RCE issue allows Remote Code Execution and XSS via SQLi.'))

remote code execution issue allow remote code execution cross site scripting via sql injection


## 5. Anwenden Pipeline (clean Variante)

In [8]:
# Beschleunigte Anwendung der Pipeline (clean Variante) mit Batch-Verarbeitung
import time
START_TIME = time.time()

# Optionale Begrenzung für schnellere Iterationen
LIMIT_ROWS = None  # z.B. 5000 für schnellen Test
BATCH_SIZE_SPACY = 400

work_df = df if LIMIT_ROWS is None else df.head(LIMIT_ROWS).copy()
texts = work_df['description'].astype(str).tolist()

normalized = [normalize_basic(t) for t in texts]
expanded = [expand_acronyms(t) for t in normalized]
# Tokenisierung simpel per split (später für spaCy Lemmas ersetzt)
token_lists = [t.split(' ') for t in expanded]

# Stopwords entfernen
filtered_tokens = [[tok for tok in toks if tok and tok not in STOPWORDS and len(tok) > 1] for toks in token_lists]

if USE_LEMMATIZATION and nlp is not None:
    # Batch Lemmatization nur auf gefilterten Tokens
    joined = [' '.join(toks) for toks in filtered_tokens]
    lemmas = []
    for doc in nlp.pipe(joined, batch_size=BATCH_SIZE_SPACY, disable=['parser','ner']):
        lemmas.append(' '.join(tok.lemma_ for tok in doc if tok.lemma_))
    final_texts = lemmas
else:
    final_texts = [' '.join(toks) for toks in filtered_tokens]

work_df['description_clean'] = final_texts

elapsed = time.time() - START_TIME
print(f'Fertig: {len(work_df)} Zeilen preprocesset in {elapsed:.2f}s (Lemmatization={USE_LEMMATIZATION and nlp is not None})')
print(work_df[['description','description_clean']].head(5))
print('Ø Tokens nach Preprocessing:', round(work_df['description_clean'].str.split().apply(len).mean(),2))

# In Haupt-df zurückschreiben
if LIMIT_ROWS is None:
    df = work_df
else:
    # Nur subset ersetzt; Hinweis ausgeben
    df.loc[work_df.index, 'description_clean'] = work_df['description_clean']
    print('WARN: LIMIT_ROWS aktiv, nur Teilmenge verarbeitet.')

Fertig: 53856 Zeilen preprocesset in 160.64s (Lemmatization=True)
                                         description  \
0  IIS 4.0 and 5.0 allows remote attackers to cau...   
1  The Microsoft .NET forms authentication capabi...   
2  Hyper-Threading technology, as used in FreeBSD...   
3  Microsoft w3wp (aka w3wp.exe) does not properl...   
4  Cross-site scripting (XSS) vulnerability in in...   

                                   description_clean  
0  iis allow remote attacker cause denial service...  
1  microsoft net form authentication capability a...  
2  hyper threading technology use freebsd operati...  
3  microsoft w3wp aka w3wp exe properly handle as...  
4  cross site script cross site script vulnerabil...  
Ø Tokens nach Preprocessing: 33.34
Ø Tokens nach Preprocessing: 33.34


## 6. Severity Encoding

In [9]:
ORDERED_SEVERITIES = ['low','medium','high','critical']
sev_norm = df['severity'].astype(str).str.lower()
unknown = sorted(set(sev_norm) - set(ORDERED_SEVERITIES))
if unknown:
    print('Verwerfe unbekannte Severity Labels:', unknown)
    before = len(df)
    df = df[sev_norm.isin(ORDERED_SEVERITIES)].copy()
    sev_norm = df['severity'].astype(str).str.lower()
    print(f'Entfernt {before - len(df)} Zeilen aufgrund unbekannter Labels.')
severity_to_id = {s:i for i,s in enumerate(ORDERED_SEVERITIES)}
severity_ids = sev_norm.map(severity_to_id)
assert not severity_ids.isna().any(), 'NaN im Severity Mapping nach Filter'
df['severity_id'] = severity_ids.astype(int)
print('Severity Mapping:', severity_to_id)
print('Verteilung:', df['severity_id'].value_counts().to_dict())

Verwerfe unbekannte Severity Labels: ['none']
Entfernt 1 Zeilen aufgrund unbekannter Labels.
Severity Mapping: {'low': 0, 'medium': 1, 'high': 2, 'critical': 3}
Verteilung: {2: 24128, 1: 20794, 3: 7497, 0: 1436}


## 7. Basisstatistik

In [10]:
report = {
    'n_rows': int(len(df)),
    'n_unique_cve': int(df['cve_id'].nunique()) if 'cve_id' in df.columns else None,
    'avg_tokens_post': float(df['description_clean'].str.split().apply(len).mean()),
}
print(df[['description_clean','severity']].sample(min(3, len(df))))
report

                                       description_clean  severity
21066  issue discover adobe acrobat reader early vers...    MEDIUM
28248  argument injection vulnerability sourcetree ma...  CRITICAL
50895  sensitive information disclosure due miss auth...    MEDIUM


{'n_rows': 53855, 'n_unique_cve': 53855, 'avg_tokens_post': 33.338130164330146}

## 8. Varianten Export

In [11]:
OUTPUT_DIR = Path('../data/processed')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
RAW_OUT = OUTPUT_DIR / 'cves_processed_text_raw.csv'
CLEAN_OUT = OUTPUT_DIR / 'cves_processed_text_clean.csv'
base_df = df[['cve_id','severity','severity_id','description_clean']].copy()

# Raw Variante erneut einlesen (minimales Cleaning + gleiche Severity-Filterlogik)
raw_src = pd.read_csv(CSV_PATH, usecols=['cve_id','severity','description']).dropna()
raw_src['severity'] = raw_src['severity'].astype(str)
raw_src_norm = raw_src['severity'].str.lower()
unknown_raw = sorted(set(raw_src_norm) - set(ORDERED_SEVERITIES))
if unknown_raw:
    before = len(raw_src)
    raw_src = raw_src[raw_src_norm.isin(ORDERED_SEVERITIES)].copy()
    raw_src_norm = raw_src['severity'].str.lower()
    print(f'Raw: entferne {before - len(raw_src)} Zeilen wegen unbekannter Severity Labels: {unknown_raw}')
severity_to_id_export = {s:i for i,s in enumerate(ORDERED_SEVERITIES)}
raw_src['severity_id'] = raw_src_norm.map(severity_to_id_export)

def minimal_clean(t: str) -> str:
    t = str(t).lower()
    t = re.sub(r'\s+', ' ', t)
    return t.strip()
raw_src['description_clean'] = raw_src['description'].apply(minimal_clean)
raw_variant = raw_src[['cve_id','severity','severity_id','description_clean']].copy()
raw_variant.to_csv(RAW_OUT, index=False)
clean_variant = base_df.copy()
clean_variant.to_csv(CLEAN_OUT, index=False)
print('Gespeichert text_raw ->', RAW_OUT.name, raw_variant.shape)
print('Gespeichert text_clean ->', CLEAN_OUT.name, clean_variant.shape)
print('Severity Mapping Export:', severity_to_id_export)
print('Hinweis: Für zusätzliche Lemma-Varianten folgt nächste Zelle.')

Raw: entferne 1 Zeilen wegen unbekannter Severity Labels: ['none']
Gespeichert text_raw -> cves_processed_text_raw.csv (53855, 4)
Gespeichert text_clean -> cves_processed_text_clean.csv (53855, 4)
Severity Mapping Export: {'low': 0, 'medium': 1, 'high': 2, 'critical': 3}
Hinweis: Für zusätzliche Lemma-Varianten folgt nächste Zelle.
Gespeichert text_raw -> cves_processed_text_raw.csv (53855, 4)
Gespeichert text_clean -> cves_processed_text_clean.csv (53855, 4)
Severity Mapping Export: {'low': 0, 'medium': 1, 'high': 2, 'critical': 3}
Hinweis: Für zusätzliche Lemma-Varianten folgt nächste Zelle.


In [12]:
# 8a. Zusätzliche Varianten mit Lemmatization (raw & clean)
# Diese Zelle erzeugt optionale Dateien mit Lemmatization für beide Pipelines.
# Sie nutzt spaCy falls verfügbar; sonst wird übersprungen.
LEMMA_RAW_OUT = OUTPUT_DIR / 'cves_processed_text_raw_lemma.csv'
LEMMA_CLEAN_OUT = OUTPUT_DIR / 'cves_processed_text_clean_lemma.csv'

if nlp is None:
    print('spaCy Modell nicht geladen -> Lemma-Varianten werden übersprungen.')
else:
    def lemmatize_text_series(series):
        docs = list(nlp.pipe(series.astype(str).tolist(), batch_size=200, disable=['parser','ner']))
        return [' '.join(tok.lemma_ for tok in doc if tok.lemma_) for doc in docs]

    # Raw Lemma
    raw_variant_lemma = raw_variant.copy()
    raw_variant_lemma['description_clean'] = lemmatize_text_series(raw_variant_lemma['description_clean'])
    raw_variant_lemma.to_csv(LEMMA_RAW_OUT, index=False)

    # Clean Lemma
    clean_variant_lemma = clean_variant.copy()
    clean_variant_lemma['description_clean'] = lemmatize_text_series(clean_variant_lemma['description_clean'])
    clean_variant_lemma.to_csv(LEMMA_CLEAN_OUT, index=False)

    print('Gespeichert text_raw_lemma ->', LEMMA_RAW_OUT.name, raw_variant_lemma.shape)
    print('Gespeichert text_clean_lemma ->', LEMMA_CLEAN_OUT.name, clean_variant_lemma.shape)

Gespeichert text_raw_lemma -> cves_processed_text_raw_lemma.csv (53855, 4)
Gespeichert text_clean_lemma -> cves_processed_text_clean_lemma.csv (53855, 4)


## 9. Token Top-Statistiken Vergleich

In [ ]:
VAR_DIR = Path('../data/processed/')
FILES = {
    'raw': VAR_DIR / 'cves_processed_text_raw.csv',
    'clean': VAR_DIR / 'cves_processed_text_clean.csv',
    'raw_lemma': VAR_DIR / 'cves_processed_text_raw_lemma.csv',
    'clean_lemma': VAR_DIR / 'cves_processed_text_clean_lemma.csv'
}

TOP_N = 10
from collections import Counter

def top_tokens(series, top_n):
    texts = series.astype(str)
    N = len(texts)
    tf_counter = Counter()
    df_counter = Counter()
    for row in texts:
        toks = row.split()
        if not toks:
            continue
        tf_counter.update(toks)
        df_counter.update(set(toks))
    mc = tf_counter.most_common(top_n)
    rows = []
    for tok, tf in mc:
        dfreq = df_counter.get(tok, 0)
        rows.append((tok, tf, dfreq, round(dfreq / N * 100, 2)))
    return pd.DataFrame(rows, columns=['token','token_freq','doc_freq','doc_pct'])

available = {k:p for k,p in FILES.items() if p.exists()}
print('Gefundene Varianten:', {k: p.name for k,p in available.items()})
variant_dfs = {}
for name, path in available.items():
    df_var = pd.read_csv(path)
    if 'description_clean' not in df_var.columns:
        print(f'WARN: description_clean fehlt in {path.name}, übersprungen')
        continue
    variant_dfs[name] = df_var

stats = {}
for name, dfv in variant_dfs.items():
    stats[name] = top_tokens(dfv['description_clean'], TOP_N)
    print(f'\nVariante: {name} (N={len(dfv)}) Top Tokens:')
    display(stats[name])

# Vergleich zwischen Paaren (raw vs clean) und (raw_lemma vs clean_lemma) falls vorhanden
from itertools import combinations
pair_specs = [('raw','clean'), ('raw_lemma','clean_lemma')]
for a,b in pair_specs:
    if a in stats and b in stats:
        merged = pd.merge(stats[a], stats[b], on='token', how='inner', suffixes=(f'_{a}', f'_{b}'))
        if not merged.empty:
            merged[f'doc_pct_delta_{b}_minus_{a}'] = merged[f'doc_pct_{b}'] - merged[f'doc_pct_{a}']
            print(f'\nGemeinsame Tokens Delta ({a} -> {b}):')
            display(merged.head(20))
        else:
            print(f'Keine gemeinsamen Top Tokens zwischen {a} und {b}.')

# Optional: Schnittmenge aller vorhandenen Varianten
if len(stats) > 1:
    common = set.intersection(*(set(dfv['token']) for dfv in stats.values())) if all('token' in dfv.columns for dfv in stats.values()) else set()
    if common:
        print(f'Gemeinsame Tokens in allen Varianten ({len(common)}):', list(sorted(list(common)))[:30])
    else:
        print('Keine gemeinsamen Tokens in allen Varianten (Top-N Ebene).')

RAW VARIANTE: Top Tokens


,token,token_freq,doc_freq,doc_pct
0,the,109805,38753,71.96
1,to,92304,44243,82.15
2,in,73463,41095,76.31
3,a,70055,36963,68.63
4,of,55980,26998,50.13
5,and,46210,25370,47.11
6,an,38119,24251,45.03
7,vulnerability,37043,21678,40.25
8,this,27847,18418,34.20
9,before,25509,13200,24.51


CLEAN VARIANTE: Top Tokens


,token,token_freq,doc_freq,doc_pct
0,vulnerability,70406,25294,46.97
1,attacker,36860,27059,50.24
2,allow,33085,30609,56.84
3,common,23233,5083,9.44
4,user,21888,16339,30.34
5,remote,19881,17736,32.93
6,open,19173,14327,26.60
7,code,18271,12871,23.90
8,via,17491,17242,32.02
9,service,17065,11503,21.36


Gemeinsame Tokens (DocFreq Delta):


,token,token_freq_raw,doc_freq_raw,doc_pct_raw,token_freq_clean,doc_freq_clean,doc_pct_clean,doc_pct_delta_clean_minus_raw
0,vulnerability,37043,21678,40.25,70406,25294,46.97,6.72
